In [1]:
%load_ext autoreload
%autoreload 2

import networkx as nx
from dotenv import load_dotenv
import os, sys
import numpy as np
from matplotlib import pyplot as plt


load_dotenv()
PROJECT_ROOT = os.getenv('PROJECT_ROOT')
# Add the project root to the Python path
sys.path.append(PROJECT_ROOT)

In [2]:
from nav_graph import load_ats_graph, load_fra_graph_into_ats_graph, route_graph_subset
import pandas as pd 

# Load the airports dataframe
airports_df = pd.read_csv(os.path.join(PROJECT_ROOT, "data", "airac", "airports.csv"))
# columns of airports_df: icao,name,latitude,longitude,elevation

In [3]:
ORIGIN_ICAO = 'EGLL'
DEST_ICAO = 'UKBB'
ORIGIN_RWY = '09R'
DEST_RWY = '18R'

# Get the latitude and longitude of the origin and destination
origin_lat = airports_df[airports_df['icao'] == ORIGIN_ICAO]['latitude'].values[0]
origin_lon = airports_df[airports_df['icao'] == ORIGIN_ICAO]['longitude'].values[0]
dest_lat = airports_df[airports_df['icao'] == DEST_ICAO]['latitude'].values[0]
dest_lon = airports_df[airports_df['icao'] == DEST_ICAO]['longitude'].values[0]
# Origin and destination airport names
origin_name = airports_df[airports_df['icao'] == ORIGIN_ICAO]['name'].values[0]
dest_name = airports_df[airports_df['icao'] == DEST_ICAO]['name'].values[0]

print(f'Origin: {ORIGIN_ICAO} - {origin_name} ({origin_lat}, {origin_lon}, {ORIGIN_RWY})')
print(f'Destination: {DEST_ICAO} - {dest_name} ({dest_lat}, {dest_lon}, {DEST_RWY})')

Origin: EGLL - HEATHROW (51.4775, -0.461389, 09R)
Destination: UKBB - BORYSPIL INTL (50.344722, 30.893333, 18R)


In [4]:
from nav_graph import generate_navigraph

route_graph = generate_navigraph(ORIGIN_ICAO, DEST_ICAO, origin_lat, origin_lon, dest_lat, dest_lon,
                                  ORIGIN_RWY, DEST_RWY,
                                  w_dct=1.0, w_fra=1.0, w_proc=0.2)


Subsetting the ATS graph to the great circle path between origin and destination...


Adding edges to subset: 100%|██████████| 25011/25011 [00:00<00:00, 916173.84it/s]

ATS graph loaded. Nodes: 1670, edges: 3817
Building FRA routing options...


Found 1037 FRA points within 100nm of the great circle path between origin and destination.          
Merging these into the ATS graph...
223 FRA points renamed for BALTIC & FRAIT & SECSI & SEE FRA


BALTIC & FRAIT & SECSI & SEE FRA: 100%|██████████| 267/267 [00:00<00:00, 584.73it/s]


...

127 FRA points renamed for UKNESFRA


UKNESFRA: 100%|██████████| 133/133 [00:00<00:00, 915.27it/s]


FRA graph merged into ATS graph. Nodes: 2666, edges: 121054
Computing cost for the ATS-FRA route graph...


Adding edges to subset: 100%|██████████| 121054/121054 [00:00<00:00, 375503.38it/s]


Adding SID and STAR graphs to the subset...
Route graph subset created. Nodes: 2922, edges: 121334
Great circle distance between origin and destination: 1179.84 nm


In [5]:
from nav_graph import plan
fpl, ccost, cdist = plan(route_graph, ORIGIN_ICAO, DEST_ICAO)

In [6]:
from utils.flightplans import format_flightplan
print(format_flightplan(fpl))


EGLL CPT5J WOD BPK Q295 BRAIN M197 REDFA DERAM L980 POLON M70 OKROT SLV SLV2J UKBB


In [7]:
from nav_graph import plan_e2e

fpl, ccost, cdist = plan_e2e(ORIGIN_ICAO, DEST_ICAO, ORIGIN_RWY, DEST_RWY,
         airports_df_path=os.path.join(PROJECT_ROOT, "data", "airac", "airports.csv"))

print('--------------------------------')
print(format_flightplan(fpl))
print('--------------------------------')
print(f"Cumulative cost: {ccost}")
print(f"Cumulative distance: {cdist}")


Origin: EGLL - HEATHROW (51.4775, -0.461389, 09R)
Destination: UKBB - BORYSPIL INTL (50.344722, 30.893333, 18R)
Subsetting the ATS graph to the great circle path between origin and destination...


Adding edges to subset: 100%|██████████| 25011/25011 [00:00<00:00, 893558.24it/s]

ATS graph loaded. Nodes: 1670, edges: 3817
Building FRA routing options...


Found 1037 FRA points within 100nm of the great circle path between origin and destination.          
Merging these into the ATS graph...
223 FRA points renamed for BALTIC & FRAIT & SECSI & SEE FRA


BALTIC & FRAIT & SECSI & SEE FRA: 100%|██████████| 267/267 [00:00<00:00, 520.97it/s]


...

127 FRA points renamed for UKNESFRA


UKNESFRA: 100%|██████████| 133/133 [00:00<00:00, 877.08it/s]


FRA graph merged into ATS graph. Nodes: 2664, edges: 121054
Computing cost for the ATS-FRA route graph...


Adding edges to subset: 100%|██████████| 121054/121054 [00:00<00:00, 304376.95it/s]


Adding SID and STAR graphs to the subset...
Route graph subset created. Nodes: 2920, edges: 121334
Great circle distance between origin and destination: 1179.84 nm
--------------------------------
EGLL CPT5J WOD BPK Q295 BRAIN M197 REDFA DERAM L980 POLON M70 OKROT SLV SLV2J UKBB
--------------------------------
Cumulative cost: 1169.5899009334014
Cumulative distance: 1234.751476976051


In [8]:
fpl

[{'from_node': 'EGLL',
  'to_node': 'CPT5J_D130B',
  'from_lat': 51.4775,
  'from_lon': -0.461389,
  'to_lat': 51.465417,
  'to_lon': -0.426283,
  'distance': 1.500043829285357,
  'cost': 0.3000087658570714,
  'edge_type': 'PROC-SID',
  'airway': 'CPT5J'},
 {'from_node': 'CPT5J_D130B',
  'to_node': 'CPT5J_D253K',
  'from_lat': 51.465417,
  'from_lon': -0.426283,
  'to_lat': 51.436861,
  'to_lon': -0.748356,
  'distance': 12.172067814529617,
  'cost': 2.434413562905924,
  'edge_type': 'PROC',
  'airway': 'CPT5J'},
 {'from_node': 'CPT5J_D253K',
  'to_node': 'CPT5J_WOD',
  'from_lat': 51.436861,
  'from_lon': -0.748356,
  'to_lat': 51.452783,
  'to_lon': -0.8788,
  'distance': 4.974109995431033,
  'cost': 0.9948219990862066,
  'edge_type': 'PROC',
  'airway': 'CPT5J'},
 {'from_node': 'CPT5J_WOD',
  'to_node': 'WOD',
  'from_lat': 51.452783,
  'from_lon': -0.8788,
  'to_lat': 51.452783,
  'to_lon': -0.8788,
  'distance': 0.0,
  'cost': 0.0,
  'edge_type': 'DCT',
  'airway': ''},
 {'from_no